In [ ]:
import tensorflow as tf
from sklearn.preprocessing import MultiLabelBinarizer
from collections import defaultdict
from tensorflow import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Flatten, Dropout, MaxPooling2D, Conv2D, BatchNormalization, GlobalAveragePooling2D
from tensorflow.keras.applications import DenseNet121, EfficientNetB0, Xception
from tensorflow.keras import layers
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
#loading training data
train_df = pd.read_csv('../input/plant-pathology-2021-fgvc8/train.csv')
train_path = "../input/plant-pathology-2021-fgvc8/train_images"
train_df.head()

In [ ]:
mlb = MultiLabelBinarizer().fit(train_df.labels.apply(lambda x : x.split()))
labels = pd.DataFrame(mlb.transform(train_df.labels.apply(lambda x : x.split())), columns = mlb.classes_)

labels = pd.concat([train_df['image'], labels], axis=1)
labels.head()

In [ ]:
image_datagen = ImageDataGenerator(
    rescale=1/255.0,
    rotation_range=5,
    zoom_range=0.1,
    shear_range=0.05,
    horizontal_flip=True,
    validation_split=0.2
    
)
IMAGE = (256, 256)
BATCH_SIZE = 32

In [ ]:
train_generator = image_datagen.flow_from_dataframe(
    labels,
    directory='../input/resized-plant2021/img_sz_256',
    x_col="image",
    y_col=labels.columns.tolist()[1:],
    target_size=IMAGE,
    color_mode="rgb",
    batch_size=BATCH_SIZE,
    subset="training",
    shuffle=True,
    seed=42,
    class_mode="raw"
)

In [ ]:
validation_generator = image_datagen.flow_from_dataframe(
    labels,
    directory='../input/resized-plant2021/img_sz_256',
    x_col="image",
    y_col=labels.columns.tolist()[1:],
    target_size=IMAGE,
    color_mode="rgb",
    batch_size=BATCH_SIZE,
    subset="validation",
    shuffle=True,
    seed=42,
    class_mode="raw"
)

In [ ]:
WEIGHTS_PATH = '../input/keras-pretrained-models/EfficientNetB0_NoTop_ImageNet.h5'
base_model = EfficientNetB0(weights=WEIGHTS_PATH,include_top=False, input_shape=(256,256,3))

x=base_model.output

x=GlobalAveragePooling2D()(x)

preds=Dense(6,activation='sigmoid')(x)

model=Model(inputs=base_model.input, outputs=preds)

In [ ]:
model.compile(
    loss="binary_crossentropy",
    optimizer="adam",
    metrics=["accuracy"])

In [ ]:
early_stopping = EarlyStopping(
    monitor='val_loss', 
    patience=5, 
    verbose=1, 
    restore_best_weights=True)

reducelr = ReduceLROnPlateau(
        monitor= 'val_loss',
        mode='min',
        factor=0.01,
        patience=2,
        verbose=0
    )

callbacks = [reducelr, early_stopping]

In [ ]:
history = model.fit(train_generator,
                validation_data=validation_generator,
                steps_per_epoch = train_generator.n // BATCH_SIZE,
                validation_steps = validation_generator.n // BATCH_SIZE,
                epochs=30,
                callbacks=callbacks)

In [ ]:
model.save('effnetb0-256.h5')